In [ ]:
!pip install sagemaker
!pip install boto3

In [13]:
import boto3

def test_aws_connection():
    sts_client = boto3.client('sts')
    # Call the get-caller-identity method to test the connection
    response = sts_client.get_caller_identity()
    print(response)

In [ ]:
test_aws_connection()

In [7]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

def get_execution_role():
    try:
        print('starting execution_role...')
        role = sagemaker.get_execution_role()
        print('starting execution_role...OK')
    except ValueError:
        print('error execution_role...')
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker-execution-role')['Role']['Arn']
        print('error execution_role...OK')

    print(f"sagemaker role arn: {role}")
    return role

def getHuggingFaceModel(role):	
	# create Hugging Face Model Class
	huggingface_model = HuggingFaceModel(
	   model_data="s3://my-ml-models-123/model_bert_sentiment_tweet_extract.tar.gz",  # path to your trained sagemaker model
	   role=role, # iam role with permissions to create an Endpoint
	   transformers_version="4.37.0", # transformers version used
	   pytorch_version="2.1.0", # pytorch version used
	   py_version="py310", # python version of the DLC
       env={"HF_TASK": "question-answering"}
	)
	return huggingface_model

def deploy(huggingface_model):
	predictor = huggingface_model.deploy(
	   initial_instance_count=1,
	   instance_type="ml.t2.medium" # The cheapest one :)
	)
	return predictor

def predict(predictor, sentiment, tweet):
    data = {
            "inputs": {
                "question": sentiment,
                "context": tweet
            }
    }
    # request
    print(predictor.predict(data))

In [3]:
role = get_execution_role()

starting execution_role...


[02/07/25 18:43:25] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=353536;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=886661;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

[02/07/25 18:43:34] WARNING  Couldn't call 'get_role' to get Role ARN from role name admin to get   ]8;id=426619;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=669199;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#5971\5971]8;;\
                             Role path.                                                                            

error execution_role...


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=416105;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=641332;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

error execution_role...OK
sagemaker role arn: arn:aws:iam::756553336060:role/sagemaker-execution-role


In [4]:
huggingface_model = getHuggingFaceModel(role)

In [5]:
predictor = deploy(huggingface_model)

[02/07/25 18:44:39] INFO     Creating model with name:                                              ]8;id=100540;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=524289;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-02-07-23-44-39-923                                 

[02/07/25 18:44:40] INFO     Creating endpoint-config with name                                     ]8;id=151578;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=128324;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#5889\5889]8;;\
                             huggingface-pytorch-inference-2025-02-07-23-44-40-845                                 

[02/07/25 18:44:41] INFO     Creating endpoint with name                                            ]8;id=64826;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=930518;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4711\4711]8;;\
                             huggingface-pytorch-inference-2025-02-07-23-44-40-845                                 

-------------------------------------------------------------------------------!

In [8]:
predict(predictor,"positive", "I really really like the song Love Story by Taylor Swift")

{'score': 0.6242777705192566, 'start': 16, 'end': 20, 'answer': 'like'}


In [9]:
predict(predictor,"negative", "So many tests todayyy I don`t feel confident about anyy.")

{'score': 0.27366480231285095, 'start': 22, 'end': 44, 'answer': 'I don`t feel confident'}


In [10]:
predict(predictor,"neutral", "is cleaning the house for her family who is comming later today..")

{'score': 0.9780168533325195, 'start': 0, 'end': 65, 'answer': 'is cleaning the house for her family who is comming later today..'}


In [11]:
predict(predictor,"positive", "im an avid fan of **** magazine and i love your magazines")

{'score': 0.13287344574928284, 'start': 6, 'end': 14, 'answer': 'avid fan'}


In [15]:
predictor.delete_endpoint()

[02/07/25 19:29:54] INFO     Deleting endpoint configuration with name:                             ]8;id=716670;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=877562;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4865\4865]8;;\
                             huggingface-pytorch-inference-2025-02-07-23-44-40-845                                 

                    INFO     Deleting endpoint with name:                                           ]8;id=457318;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=481318;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4855\4855]8;;\
                             huggingface-pytorch-inference-2025-02-07-23-44-40-845                                 